# NJMLS

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import json
import math
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from IPython.core.display import display, HTML
from concurrent.futures import ThreadPoolExecutor

pd.set_option('display.max_rows', None)

def preview_njmls_df(mydf, save=False):
    columns = ['address', 'city', 'county', 'bedrooms', 'baths_full', 'baths_part',
               'price', 'tax', 'style', 'rooms', 'list_date', 'park_and_ride_name',
               'park_and_ride_duration_text', 'park_and_ride_type', 'park_and_ride_distance',
               'park_and_ride_bus_name', 'park_and_ride_bus_distance',
               'barnabas_duration_text', 'lat', 'lng',]
    if save:
        mydf[columns].to_csv('njmls.csv')
    display(HTML(
        """<style>.dataframe td {white-space: nowrap;}</style>""" +
        mydf[columns].to_html(
            formatters={
                '__index__':
                    lambda id: f'<a target="_blank" href="https://www.njmls.com/listings/index.cfm?action=dsp.info&mlsnum={id}">{id}</a>'},
            escape=False)))

**Import global settings**

In [3]:
from settings import (blacklist_cities, counties, ONE_HOUR, HALF_HOUR, TEN_MINUTES,
                      MIN_PRICE, MAX_PRICE, MAX_TAX, MIN_BEDS, MIN_BATHS,)

print(MIN_PRICE, MAX_PRICE, MAX_TAX, MIN_BEDS, MIN_BATHS,)

300000 525000 16000 3 2


**Download listings from [NJMLS](http://www.njmls.com/)**

In [4]:
from njmls import get_listings, get_listing_detail

def get_listing_detail_wrapper(listing):
    listing_detail = get_listing_detail(listing['id'])
    listing_detail['lat'] = listing['lat']
    listing_detail['lng'] = listing['lng']
    return listing_detail
    
listings = []
for county in tqdm(counties):

    current_listings1 = list(get_listings(
            min_beds=MIN_BEDS,
            min_baths=MIN_BATHS,
            county_search=True,
            min_price=MIN_PRICE,
            max_price=int(math.floor((MAX_PRICE+MIN_PRICE)/2)),
            counties=[county.upper()],
            proptypes=['1']))

    current_listings2 = list(get_listings(
            min_beds=3,
            min_baths=2,
            county_search=True,
            min_price=int(math.ceil((MAX_PRICE+MIN_PRICE)/2)),
            max_price=MAX_PRICE,
            counties=[county.upper()],
            proptypes=['1']))

    current_listings = current_listings1 + current_listings2

    listing_ids = [x['id'] for x in current_listings]
    listings_dict = {x['id']:x for x in current_listings}

    with ThreadPoolExecutor(max_workers=20) as e:
        listings += tqdm(e.map(get_listing_detail_wrapper, current_listings), total=len(current_listings))

print(f"Downloaded {len(listings)} listings from NJMLS.")

listings = [x for x in listings if x['address']]
print(f"Filtered down to {len(listings)} listings due to null address.")


HBox(children=(IntProgress(value=0, max=8), HTML(value='')))

HBox(children=(IntProgress(value=0, max=211), HTML(value='')))

HBox(children=(IntProgress(value=0, max=19), HTML(value='')))

HBox(children=(IntProgress(value=0, max=70), HTML(value='')))

HBox(children=(IntProgress(value=0, max=6), HTML(value='')))

HBox(children=(IntProgress(value=0, max=38), HTML(value='')))

HBox(children=(IntProgress(value=0, max=370), HTML(value='')))

HBox(children=(IntProgress(value=0, max=10), HTML(value='')))

HBox(children=(IntProgress(value=0, max=15), HTML(value='')))


Downloaded 739 listings from NJMLS.
Filtered down to 717 listings due to null address.


**Geocode each address using Google Maps API**

In [5]:
from geolocate import add_geocode_to_listing
for listing in tqdm(listings):
    if not listing.get('address'): continue
    add_geocode_to_listing(listing)

HBox(children=(IntProgress(value=0, max=717), HTML(value='')))

could not find county for 1742511 46 Veranda Ave, North Caldwell, NJ, USA [{'long_name': '46', 'short_name': '46', 'types': ['street_number']}, {'long_name': 'Veranda Avenue', 'short_name': 'Veranda Ave', 'types': ['route']}, {'long_name': 'North Caldwell', 'short_name': 'North Caldwell', 'types': ['locality', 'political']}, {'long_name': 'New Jersey', 'short_name': 'NJ', 'types': ['administrative_area_level_1', 'political']}, {'long_name': 'United States', 'short_name': 'US', 'types': ['country', 'political']}]
could not find county for 1745962 790 Grove St, Clifton, NJ 07013, USA [{'long_name': '790', 'short_name': '790', 'types': ['street_number']}, {'long_name': 'Grove Street', 'short_name': 'Grove St', 'types': ['route']}, {'long_name': 'Clifton', 'short_name': 'Clifton', 'types': ['locality', 'political']}, {'long_name': 'New Jersey', 'short_name': 'NJ', 'types': ['administrative_area_level_1', 'political']}, {'long_name': 'United States', 'short_name': 'US', 'types': ['country',

**Find closest NJ Transit Park and Ride to each address**

In [6]:
from commute import add_closest_park_and_ride_to_listing

for i in tqdm(range(len(listings))):
    listing = listings[i]
    add_closest_park_and_ride_to_listing(listing)

HBox(children=(IntProgress(value=0, max=717), HTML(value='')))

**Get commute time for each address through both park and ride and walking**

In [7]:
from commute import add_commute_to_listing

for i in tqdm(range(len(listings))):
    listing = listings[i]
    add_commute_to_listing(listing)

print(sum(1 for listing in listings if listing.get('park_and_ride_duration')),
      "out of", len(listings), "have park and ride commute times.")

print(sum(1 for listing in listings if listing.get('nyc_duration')),
  "out of", len(listings), "have walking commute times.")

HBox(children=(IntProgress(value=0, max=717), HTML(value='')))


716 out of 717 have park and ride commute times.
649 out of 717 have walking commute times.


In [8]:
df = pd.DataFrame(listings)
df.set_index('id', inplace=True)
df = df[
    (~df.city.isin(blacklist_cities)) &
    (~df.basement.str.contains('Crawlspace')) &
    (df.basement != 'None') &
    (df['style'] != 'Cape Cod') &
    (df.barnabas_duration < HALF_HOUR + TEN_MINUTES) &
    (df.park_and_ride_duration1 < TEN_MINUTES) &
    (df.park_and_ride_duration < ONE_HOUR + TEN_MINUTES)
]
df = df.sort_values(by=['city',], ascending=[True])
print(len(df), "listings after filtering")
preview_njmls_df(df)

210 listings after filtering


,address,city,county,bedrooms,baths_full,baths_part,price,tax,style,rooms,list_date,park_and_ride_name,park_and_ride_duration_text,park_and_ride_type,park_and_ride_distance,park_and_ride_bus_name,park_and_ride_bus_distance,barnabas_duration_text,lat,lng
id,,,,,,,,,,,,,,,,,,,,
1748481,89 Mountain Avenue,Bloomfield,Essex,3,2,0,399000,13884.0,Colonial,6,12/21/2017,Watchung Avenue,"6 mins, 45 mins",rail,1.141127,Allwood Road,1.494435,25 mins,40.8337,-74.1859
1748219,151 Sadler Road,Bloomfield,Essex,3,2,0,370000,11603.0,Colonial,7,12/17/2017,Clifton Commons,"8 mins, 47 mins",bus,1.672280,Clifton Commons,1.672280,23 mins,40.8262,-74.173
1741304,85 Bellevue Terrace,Bloomfield,Essex,3,2,1,424900,12668.0,Split Level,8,10/13/2017,Allwood Road,"4 mins, 33 mins",bus,1.063173,Allwood Road,1.063173,27 mins,40.8408,-74.186
1736975,15 George Street,Bloomfield,Essex,4,2,1,439000,14719.0,Colonial,8,09/12/2017,Watchung Avenue,"6 mins, 45 mins",rail,1.390126,Allwood Road,1.715707,22 mins,40.8293,-74.1804
1742512,10 Claremont Avenue,Bloomfield,Essex,4,2,2,479800,16663.0,Colonial,11,10/18/2017,Watchung Avenue,"6 mins, 45 mins",rail,1.176241,Allwood Road,1.444312,25 mins,40.8344,-74.1854
1735422,338 Essex Avenue,Bloomfield,Essex,5,2,1,439900,16232.0,Colonial,9,09/01/2017,Walnut Street,"5 mins, 42 mins",rail,0.751703,Allwood Road,2.776278,24 mins,40.8166,-74.1953
1730541,41 Clark Avenue,Bloomfield,Essex,3,2,0,309000,9927.0,Colonial,7,07/26/2017,Walnut Street,"5 mins, 42 mins",rail,0.927320,Clifton Commons,2.847465,23 mins,40.8138,-74.1925
1746287,175 Jerome Place,Bloomfield,Essex,4,3,0,459000,8715.0,Colonial,8,11/28/2017,Bloomfield,"5 mins, 36 mins",rail,0.632988,Clifton Commons,3.454818,21 mins,40.7919,-74.188
1742305,128 Willow Street,Bloomfield,Essex,3,3,0,349000,10124.0,Colonial,7,10/23/2017,Watsessing Avenue,"3 mins, 33 mins",rail,0.293689,Newark Penn Station Bus Lanes,4.057832,21 mins,40.7844,-74.2036


In [10]:
import folium

home = (40.8961863, -74.1726829,)

m = folium.Map(location=home, zoom_start=12)

folium.Marker(home, tooltip=f'<i>115 Dumont Ave. Clifton, NJ</i>', icon=folium.Icon(color='green')).add_to(m)

for id, row in df.iterrows():
    url = f"http://www.njmls.com/listings/index.cfm?action=dsp.info&mlsnum={id}"
    address = row.address + ", " + row.city
    popup = f"""
    Address: {address}<br/>
    MLS: <a target="_blank" href="{url}">{id}</a><br/>
    Price: {row.price}<br/>
    Tax: {row.tax}<br/>
    Dad (Park and Ride): {row.park_and_ride_duration_text}
    Dad (Walk and Ride): {row.nyc_duration_text}
    Mom: {row.barnabas_duration_text}
    """
    folium.Marker((row.lat, row.lng,), popup=f'<i>{popup}</i>', tooltip=f'<i>{address}</i>').add_to(m)

m